In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

import time

In [11]:
chrome_options = Options()
chrome_options.add_argument("--headless")  

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)


In [3]:
import pandas as pd
df=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

In [4]:
df.head()

,Label,HD Text,AR Text
0,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 1: 193549069 to 193549087GenBankGraphics...,Query 3 GCAGTAGCTAGCTAGCTAG 21\n ...
1,"Homo sapiens chromosome 3, GRCh38.p14 Primary ...",Range 2: 81734675 to 81734691GenBankGraphics\n...,Query 4 CAGTAGCTAGCTAGCTA 20\n ...
2,"Homo sapiens isolate CHM13 chromosome 2, alter...",Range 3: 1015481 to 1015495GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
3,"Homo sapiens isolate CHM13 chromosome 6, alter...",Range 4: 2872649 to 2872663GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
4,"Homo sapiens isolate CHM13 chromosome 21, alte...",Range 5: 26308090 to 26308104GenBankGraphics\n...,Query 7 TAGCTAGCTAGCTAG 21\n ...


In [ ]:
print

In [54]:
print(df.loc[0,'HD Text'])


Range 1: 193549069 to 193549087GenBankGraphics
Next Match
Previous Match
Alignment statistics for match #1
Score Expect Identities Gaps Strand
38.2 bits(19) 0.084 19/19(100%) 0/19(0%) Plus/Minus
Features:
84024 bp at 5' side: interleukin-1 receptor accessory protein isoform x3
122539 bp at 3' side: geminin coiled-coil domain-containing protein 1


In [55]:
print(df.loc[0,'AR Text'])

Query  3          GCAGTAGCTAGCTAGCTAG  21
                  |||||||||||||||||||
Sbjct  193549087  GCAGTAGCTAGCTAGCTAG  193549069


In [5]:
import pandas as pd

df = pd.read_csv('blast_results.csv')

ranges = []
alignment_stats = []
features = []
queries = []
subjects = []

for index, row in df.iterrows():
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    range_info = hd_lines[0].replace('GenBankGraphics', '').strip()
    ranges.append(range_info)

    alignment_stat_info = hd_lines[3:6]
    alignment_stats.append(' '.join(alignment_stat_info).strip())

    features_info = hd_lines[7:]
    features.append(' '.join(features_info).strip())

    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    query_info = ar_lines[0].strip()
    subject_info = ar_lines[2].strip()

    queries.append(query_info)
    subjects.append(subject_info)

new_df = pd.DataFrame({
    'Range': ranges,
    'Alignment Statistics': alignment_stats,
    'Features': features,
    'Query': queries,
    'Subject': subjects
})

new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [6]:
df2=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results_split.csv')

In [7]:
df2.head()

,Range,Alignment Statistics,Features,Query,Subject
0,Range 1: 193549069 to 193549087,Alignment statistics for match #1 Score Expect...,84024 bp at 5' side: interleukin-1 receptor ac...,Query 3 GCAGTAGCTAGCTAGCTAG 21,Sbjct 193549087 GCAGTAGCTAGCTAGCTAG 193549069
1,Range 2: 81734675 to 81734691,First Match Alignment statistics for match #2 ...,"Features: 1,4-alpha-glucan-branching enzyme",Query 4 CAGTAGCTAGCTAGCTA 20,Sbjct 81734675 CAGTAGCTAGCTAGCTA 81734691
2,Range 3: 1015481 to 1015495,First Match Alignment statistics for match #3 ...,Features: 617909 bp at 5' side: neural cell ad...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 1015481 TAGCTAGCTAGCTAG 1015495
3,Range 4: 2872649 to 2872663,First Match Alignment statistics for match #4 ...,Features: contactin-4 isoform x1 contactin-4 i...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 2872663 TAGCTAGCTAGCTAG 2872649
4,Range 5: 26308090 to 26308104,First Match Alignment statistics for match #5 ...,Features: 510830 bp at 5' side: 3-oxoacyl-[acy...,Query 7 TAGCTAGCTAGCTAG 21,Sbjct 26308104 TAGCTAGCTAGCTAG 26308090


In [8]:
import pandas as pd
import re

df = pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

sequences = []
starts = []
ends = []
phenotypes = []

for index, row in df.iterrows():
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    range_info = hd_lines[0]
    feature_lines = hd_lines[-2:]  

    range_match = re.search(r'Range \d+: (\d+) to (\d+)', range_info)
    if range_match:
        start = range_match.group(1)
        end = range_match.group(2)
        starts.append(start)
        ends.append(end)
    else:
        starts.append(None)
        ends.append(None)

    phenotypes_info = [line.split(': ')[1] for line in feature_lines if ': ' in line]
    phenotypes.append(', '.join(phenotypes_info))

    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    sequence_info = ar_lines[0].strip()
    sequence_match = re.search(r'Query \d+\s+(\w+)\s+\d+', sequence_info)
    if sequence_match:
        sequence = sequence_match.group(1)
        sequences.append(sequence)
    else:
        sequences.append(None)

new_df = pd.DataFrame({
    'sequence': sequences,
    'start': starts,
    'end': ends,
    'phenotypes': phenotypes
})

new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [61]:
new_df.head()

,sequence,start,end,phenotypes
0,None,193549069,193549087,interleukin-1 receptor accessory protein isofo...
1,None,81734675,81734691,
2,None,1015481,1015495,neural cell adhesion molecule l1-like protein ...
3,None,2872649,2872663,
4,None,26308090,26308104,"3-oxoacyl-[acyl-carrier-protein] synthase, mit..."


In [10]:
import pandas as pd
import re

df = pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results.csv')

chromosome_location=[]
sequences = []
starts = []
ends = []
phenotypes = []

for index, row in df.iterrows():
    hd_text = row['HD Text']
    hd_lines = hd_text.split('\n')

    range_info = hd_lines[0]
    feature_lines = hd_lines[-2:]  

    range_match = re.search(r'Range \d+: (\d+) to (\d+)', range_info)
    if range_match:
        start = range_match.group(1)
        end = range_match.group(2)
        starts.append(start)
        ends.append(end)
    else:
        starts.append(None)
        ends.append(None)

    phenotypes_info = [line.split(': ')[1] for line in feature_lines if ': ' in line]
    phenotypes.append(', '.join(phenotypes_info))

    ar_text = row['AR Text']
    ar_lines = ar_text.split('\n')

    query_line = ar_lines[0].strip()
    query_match = re.search(r'Query\s+\d+\s+([A-Z]+)\s+\d+', query_line)
    if query_match:
        sequence = query_match.group(1)
        sequences.append(sequence)
    else:
        sequences.append(None)
    chromosome_location.append(row['Label'])


new_df = pd.DataFrame({
    'chromosome location': chromosome_location,
    'sequence': sequences,
    'start': starts,
    'end': ends,
    'phenotypes': phenotypes
})

new_df.to_csv('blast_results_split.csv', index=False)

print("New CSV file 'blast_results_split.csv' created successfully.")


New CSV file 'blast_results_split.csv' created successfully.


In [63]:
df3=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_results_split.csv')

In [64]:
df3.head()

,chromosome location,sequence,start,end,phenotypes
0,"Homo sapiens isolate CHM13 chromosome 3, alter...",GCAGTAGCTAGCTAGCTAG,193549069,193549087,interleukin-1 receptor accessory protein isofo...
1,"Homo sapiens chromosome 3, GRCh38.p14 Primary ...",CAGTAGCTAGCTAGCTA,81734675,81734691,NaN
2,"Homo sapiens isolate CHM13 chromosome 2, alter...",TAGCTAGCTAGCTAG,1015481,1015495,neural cell adhesion molecule l1-like protein ...
3,"Homo sapiens isolate CHM13 chromosome 6, alter...",TAGCTAGCTAGCTAG,2872649,2872663,NaN
4,"Homo sapiens isolate CHM13 chromosome 21, alte...",TAGCTAGCTAGCTAG,26308090,26308104,"3-oxoacyl-[acyl-carrier-protein] synthase, mit..."


In [65]:
print(df3)

                                 chromosome location             sequence  \
0  Homo sapiens isolate CHM13 chromosome 3, alter...  GCAGTAGCTAGCTAGCTAG   
1  Homo sapiens chromosome 3, GRCh38.p14 Primary ...    CAGTAGCTAGCTAGCTA   
2  Homo sapiens isolate CHM13 chromosome 2, alter...      TAGCTAGCTAGCTAG   
3  Homo sapiens isolate CHM13 chromosome 6, alter...      TAGCTAGCTAGCTAG   
4  Homo sapiens isolate CHM13 chromosome 21, alte...      TAGCTAGCTAGCTAG   

       start        end                                         phenotypes  
0  193549069  193549087  interleukin-1 receptor accessory protein isofo...  
1   81734675   81734691                                                NaN  
2    1015481    1015495  neural cell adhesion molecule l1-like protein ...  
3    2872649    2872663                                                NaN  
4   26308090   26308104  3-oxoacyl-[acyl-carrier-protein] synthase, mit...  


second try

In [2]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE_TYPE=BlastSearch&BLAST_SPEC=OGP__9606__9558&LINK_LOC=blasthome"

driver = webdriver.Chrome()

try:
    driver.get(url)

    sequence_textarea = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "seq"))
    )
    sequence_textarea.send_keys("ATGCAGTAGCTAGCTAGCTAG")  

    blast_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//div[@id='blastButton1']/input[@class='blastbutton']"))
    )
    blast_button.click()

    format_organism_input = WebDriverWait(driver, 120).until(
        EC.presence_of_element_located((By.ID, "qorganism"))
    )

    link_to_click = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//tr[@ind='1']//a[@class='deflnDesc']"))
    )
    link_to_click.click()

    hd_texts = []
    ar_texts = []
    labels = []

    hd_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[starts-with(@id, 'hd_')]"))
    )
    ar_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[starts-with(@id, 'ar_')]/pre"))
    )

    label_div_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, "//div[@class='dlfRow']"))
    )

    num_entries = min(len(hd_div_elements), len(ar_div_elements), len(label_div_elements))

    for i in range(num_entries):
        try:
            label_text = label_div_elements[i].text
            hd_texts.append(hd_div_elements[i].text)
            ar_texts.append(ar_div_elements[i].text)
            labels.append(label_text)
        except Exception as e:
            print(f"An error occurred while fetching entry {i}: {e}")
            break

    combined_texts = zip(labels, hd_texts, ar_texts)

    with open('blast_results.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Label', 'HD Text', 'AR Text']) 
        for label, hd_text, ar_text in combined_texts:
            csvwriter.writerow([label, hd_text, ar_text])

finally:
    time.sleep(10)  
    driver.quit()


In [1]:
import pandas as pd

In [2]:
df=pd.read_csv(r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_.csv')

In [3]:
df.head()

,dlfRow,hd,ar
0,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 1: 193549069 to 193549087GenBankGraphics...,Query 3 GCAGTAGCTAGCTAGCTAG 21\n ...
1,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 2: 81734675 to 81734691GenBankGraphics\n...,Query 4 CAGTAGCTAGCTAGCTA 20\n ...
2,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 3: 1015481 to 1015495GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
3,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 4: 2872649 to 2872663GenBankGraphics\nNe...,Query 7 TAGCTAGCTAGCTAG 21\n ...
4,"Homo sapiens isolate CHM13 chromosome 3, alter...",Range 5: 26308090 to 26308104GenBankGraphics\n...,Query 7 TAGCTAGCTAGCTAG 21\n ...


In [4]:
import csv
import re

def parse_hd(hd_text):
    match = re.search(r'Range \d+: (\d+) to (\d+)', hd_text)
    if match:
        position_start = int(match.group(1))
        position_end = int(match.group(2))
    else:
        position_start = None
        position_end = None
    
    features = []
    feature_lines = hd_text.split('Features:')[-1].strip().split('\n')
    for line in feature_lines:
        if line.strip():
            features.append(line.strip())
    
    return position_start, position_end, features

def parse_ar(ar_text):
    match = re.search(r'Query\s+\d+\s+(\S+)\s+\d+\s+\|\|+\s+(\S+)\s+\d+', ar_text)
    if match:
        query_seq = match.group(1)
        sbjct_seq = match.group(2)
        sequence_alignment = query_seq + ' | ' + sbjct_seq
    else:
        sequence_alignment = None

    return sequence_alignment

def transform_csv(input_csv, output_csv):
    with open(input_csv, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        results = []
        
        for row in reader:
            dlfRow = row['dlfRow']
            hd = row['hd']
            ar = row['ar']
            
            position_start, position_end, features = parse_hd(hd)
            sequence_alignment = parse_ar(ar)
            features_str = '; '.join(features)
            
            results.append({
                'dlfRow': dlfRow,
                'sequence_alignment': sequence_alignment,
                'position_start': position_start,
                'position_end': position_end,
                'features': features_str
            })
    
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['dlfRow', 'sequence_alignment', 'position_start', 'position_end', 'features']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for result in results:
            writer.writerow(result)

input_csv = r'C:\Users\Anis\Desktop\Crispr model\off_target\blast_.csv'
output_csv = 'transformed_blast.csv'
transform_csv(input_csv, output_csv)
